## Visualization of Football Matches using the Lean Startup Method (part 2)

This is the second part of the [visualization_football_part1](visualization_football.ipynb).


In [28]:
#Load all libraries
import os,sys  
import pandas as pd
import numpy as np
import datashader as ds
import datashader.transfer_functions as tf
from datashader import reductions
from datashader.colors import colormap_select, Hot, inferno
from datashader.bokeh_ext import InteractiveImage
from bokeh.palettes import Greens3, Blues3, Blues4, Blues9, Greys9
from bokeh.plotting import figure, output_notebook
from bokeh.tile_providers import WMTSTileSource, STAMEN_TONER, STAMEN_TERRAIN
import math
from difflib import SequenceMatcher
from utils import country_dict, to_web_marcator, aggregate_dataframe_coordinates

output_notebook()
print(sys.version)

Loading BokehJS ...

3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]


Now that we have the map, we can start to improve it. If you are into football, you will notice that there are several points in the north of Spain, that corresponds to Sporting de Gijon. Sadly for Sporting supporters, they have never reached to the Champions. Instead, Sporting Clube de Portugal has participated several times in the championship, but since the current dataset doesn't have teams from Portugal, the system mistakenly thinks that `Sporting CP` from `champions.csv` is the Sporting de Gijon from `stadiums.csv`. So lets fix this issue by getting the stadiums coordinates from the rest of the countries in Europe.  
We can get that info from [wikidata](https://query.wikidata.org/). Using `SPARQL` language we can get the information we need:
```SQL
SELECT ?clubLabel ?venueLabel ?coordinates ?countryLabel  WHERE {
  ?club wdt:P31 wd:Q476028.
  ?club wdt:P115 ?venue.
  ?venue wdt:P625 ?coordinates.
  ?club wdt:P17 ?country.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ?clubLabel
```
This generates 4435 Results in 10111 ms that can be saved to a csv file.

In [12]:
df_stadium_read = pd.read_csv('stadiums_wikidata.csv', usecols=['clubLabel','venueLabel','coordinates','countryLabel'])
#df_stadium_read.tail()

The first step is to clean the column coordinates. For that we will use a regex pattern. The pattern `[-+]?[0-9]*\.?[0-9]+` finds any signed float in a string. Then we create two patterns separated by a space and name the columns using this format: `(?P<Longitude>)`. Finally we have to concatente the club information with the coordinates.

In [13]:
df_temp = df_stadium_read['coordinates'].str.extract('(?P<Longitude>[-+]?[0-9]*\.?[0-9]+) (?P<Latitude>[-+]?[0-9]*\.?[0-9]+)', expand=True)
df_stadium_new = pd.concat([df_stadium_read['clubLabel'],df_stadium_read['venueLabel'], df_temp, df_stadium_read['countryLabel']], axis=1) 
df_stadium_new = df_stadium_new.rename(columns = {'clubLabel':'Team', 'venueLabel':'Stadium','countryLabel':'Country'})
print("Number of rows: %d" % df_stadium_new.shape[0])
unique_teams_stadium = list(set(df_stadium_new['Team']))
print("Unique team's name number: %d" % len(unique_teams_stadium))
df_stadium_new.take(list(range(3388,3393)))

Number of rows: 4435
Unique team's name number: 4314


,Team,Stadium,Longitude,Latitude,Country
3388,Real Madrid,Juan Ramón Loubriel Stadium,-66.150833,18.393333,United States of America
3389,Real Madrid,Galeshewe Stadium,24.740857,-28.716786,South Africa
3390,Real Madrid C,Ciudad Real Madrid,-3.61166667,40.47916667,Spain
3391,Real Madrid C.F.,Ciudad Real Madrid,-3.61166667,40.47916667,Spain
3392,Real Madrid FC,Estadio Santiago Bernabéu,-3.68835,40.45306,Spain


As it can be seen in the previous dataframe, we came into another problem. The new dataset contains all team instances, all over the world. There are some teams that have the same name in different countries, there is a Real Madrid team from USA and South Africa, and a similar name within a country. In our case, we are only interested in the instance `Real Madrid, Estadio Santiago Bernabeu, -3.68835, 40.45306, Spain`. So how can we find an automated way to filter the correct teams that have participated in the Champions League?

A practical approach is to combine an automated and manual way. With the data we have so far we can automatically filter the two first entries using the country. We can get the country info from `champions.csv` dataset. To distinguish teams from the same country we will filter them manually. 

The first step then is to get a dataframe with all the teams that have participated in Champions and their country of origin. Then we have to remove the repeated entries and rename the country code to the country name that can be found in wikidata. 

In [14]:
df_match = pd.read_csv('champions.csv', usecols=['Date','home','visitor','hcountry','vcountry'])
df_match = df_match.rename(columns = {'hcountry':'home_country', 'vcountry':'visitor_country'})
df_match_home = df_match[['home','home_country']].rename(columns={'home':'Team','home_country':'Country'})
df_match_visitor = df_match[['visitor','visitor_country']].rename(columns={'visitor':'Team','visitor_country':'Country'})
df_champions_teams = pd.concat([df_match_home,df_match_visitor], axis=0, ignore_index=True)
df_champions_teams = df_champions_teams.drop_duplicates()
print("Number of unique teams: %d" % df_champions_teams.shape[0])
df_champions_teams['Country'].replace(country_dict, inplace=True)
#df_champions_teams.to_csv('match_unique.csv')# To check that the mapping is correct
df_champions_teams.sort_values(by='Team',inplace=True)
df_champions_teams = df_champions_teams.reset_index(drop=True)
#df_champions_teams.head()

Number of unique teams: 502


Once we have the list of all teams that have participated in the Champions League, we have to generate a new dataset relating each Champions League matches with the coordinates of the team stadiums. For that we will use the function `similar` to match a the name of the team in the different datasets similarly as we did before. 

Once the csv has been generated, let's manually erase the combinations that are not correct and save everything in a new file. We won't correct those entries that are not matched, a Data Science project is better out than perfect!! 

In [15]:
%%time
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

def get_info_similar_team_country(team, country, df_stadium, df, threshold, verbose):
    team2 = "Unknown"
    stadium = "Unknown"
    latitude = np.NaN
    longitude = np.NaN
    cols = list(df)
    for idx, val in enumerate(df_stadium['Team']):
        rank = similar(team, val)
        if rank > threshold and country == df_stadium['Country'].iloc[idx]:
            if(verbose): print("%s and %s(Idx=%d) are %f similar and from the same country %s." 
                               % (team, val, idx, rank, country))
            team2 = df_stadium['Team'].iloc[idx]
            stadium = df_stadium['Stadium'].iloc[idx]
            latitude = df_stadium['Latitude'].iloc[idx]
            longitude = df_stadium['Longitude'].iloc[idx]
            dtemp = pd.DataFrame([[team, team2, stadium, latitude, longitude, country]], columns=cols)
            df = df.append(dtemp, ignore_index=True)
    #if there is no match, register it
    if(team2 == "Unknown"):
        df_nomatch = pd.DataFrame([[team, team2, stadium, latitude, longitude, country]], columns=cols)
        df = df.append(df_nomatch, ignore_index=True)
    return df

def generate_new_stadium_dataset(df_champions_teams, df_stadium_new, threshold=0.6, verbose=False):
    df = pd.DataFrame(columns=['Team', 'Team2', 'Stadium', 'Latitude','Longitude','Country'])
    for idx, row in df_champions_teams.iterrows():
        df = get_info_similar_team_country(row['Team'],row['Country'], df_stadium_new, df, 
                                           threshold=threshold, verbose=verbose)
    return df

verbose = False # You can change this to True to see all the combinations
threshold = 0.5
df_stadiums_champions = generate_new_stadium_dataset(df_champions_teams, df_stadium_new, threshold, verbose)
df_stadiums_champions.to_csv('stadiums_champions.csv', index=False)

CPU times: user 2min 9s, sys: 80 ms, total: 2min 9s
Wall time: 2min 9s


After we filtered the entries in the csv, let's load again the data and repeat the process.

In [16]:
df_stadiums_champions = pd.read_csv('stadiums_champions_filtered.csv', usecols=['Team','Stadium','Latitude','Longitude','Country'])
#df_stadiums_champions.head()

As in part 1, let's create a dataframe that relates each match with the coordinates of its stadium. 

In [17]:
%%time
def get_info_similar_team(team, df_stadium, threshold=0.6, verbose=False):
    max_rank = 0
    max_idx = -1
    stadium = "Unknown"
    latitude = np.NaN
    longitude = np.NaN
    for idx, val in enumerate(df_stadium['Team']):
        rank = similar(team, val)
        if rank > threshold:
            if(verbose): print("%s and %s(Idx=%d) are %f similar." % (team, val, idx, rank))
            if rank > max_rank:
                if(verbose): print("New maximum rank: %f" %rank)
                max_rank = rank
                max_idx = idx
                stadium = df_stadium['Stadium'].iloc[max_idx]
                latitude = df_stadium['Latitude'].iloc[max_idx]
                longitude = df_stadium['Longitude'].iloc[max_idx]
    return stadium, latitude, longitude

df_match_stadium_new= df_match
home_stadium_index = df_match_stadium_new['home'].map(lambda x: get_info_similar_team(x, df_stadiums_champions))
visitor_stadium_index = df_match_stadium_new['visitor'].map(lambda x: get_info_similar_team(x, df_stadiums_champions))
df_home = pd.DataFrame(home_stadium_index.tolist(), columns=['home_stadium', 'home_latitude', 'home_longitude'])
df_visitor = pd.DataFrame(visitor_stadium_index.tolist(), columns=['visitor_stadium', 'visitor_latitude', 'visitor_longitude'])
df_match_stadium_new = pd.concat([df_match_stadium_new, df_home, df_visitor], axis=1, ignore_index=False)
df1 = df_match_stadium_new['home_stadium'] == 'Unknown'
df2 = df_match_stadium_new['visitor_stadium'] == 'Unknown'
n_complete_matches = df_match_stadium_new.shape[0] - df_match_stadium_new[df1 | df2].shape[0]
print("Number of matches with complete data: %d out of %d" % (n_complete_matches, df_match_stadium_new.shape[0]))
df_match_stadium_new.take([0,1])

Number of matches with complete data: 5110 out of 6554
CPU times: user 5min 24s, sys: 268 ms, total: 5min 24s
Wall time: 5min 24s


The next step is to aggregate the coordinates and transform them to mercator format. 

In [18]:
df_agg = aggregate_dataframe_coordinates(df_match_stadium_new)

In [19]:
df_agg_mercator = df_agg.apply(lambda row: to_web_mercator(row['Latitude'], row['Longitude']), axis=1)
print("Number of rows: %d" % df_agg_mercator.shape[0])
df_agg_mercator.head()

Number of rows: 15330


,Latitude,Longitude
0,-1.019774e+06,4.687522e+06
1,2.277504e+06,5.588304e+06
2,NaN,NaN
3,6.821102e+05,5.808884e+06
4,-4.105852e+05,4.932000e+06


The next step is to plot the trayectories in the map using datashader

In [20]:
plot_width  = 850
plot_height = 600
x_range = (-1.9e6, 5.9e6)
y_range = (3.7e6, 9.0e6)
def create_image(x_range=x_range, y_range=y_range, w=plot_width, h=plot_height):
    cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)
    agg = cvs.line(df_agg_mercator, 'Latitude', 'Longitude',  ds.count())
    #img = tf.shade(agg, cmap=reversed(Blues3), how='eq_hist')
    img = tf.shade(agg, cmap=inferno, how='eq_hist')
    return img

def base_plot(tools='pan,wheel_zoom,reset',plot_width=plot_width, plot_height=plot_height,**plot_args):
    p = figure(tools=tools, plot_width=plot_width, plot_height=plot_height,
        x_range=x_range, y_range=y_range, outline_line_color=None,
        min_border=0, min_border_left=0, min_border_right=0,
        min_border_top=0, min_border_bottom=0, **plot_args)
    
    p.axis.visible = False
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    
    return p

In [21]:
ArcGIS=WMTSTileSource(url='http://server.arcgisonline.com/ArcGIS/rest/services/World_Street_Map/MapServer/tile/{Z}/{Y}/{X}.png')
p = base_plot()
p.add_tile(ArcGIS)
#InteractiveImage(p, create_image)

Finally the image for github
<p align="center">
<img src="map2.JPG" alt="Matches between teams in the Champions League" width="60%"/>
</p>